In [19]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import os
import random
import tqdm

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import classification_report, confusion_matrix
from functools import partial
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import *
from tensorflow.keras import backend as K
from scipy.spatial.distance import euclidean
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.model_selection import train_test_split

sns.set_theme()

### Functions

In [48]:
# split a univariate sequence into samples
def split_sequence(sequence, window_size, step):
    X = list()
    for i in range(0, len(sequence), step):
        # find the end of this pattern
        end_ix = i + window_size
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x = sequence[i:end_ix]
        X.append(seq_x)
    return np.array(X)

def generate_pairs(data, labels, window_size, n_features):
    ### GENERATE PAIR SAMPLES FOR SIAMESE NETWORK ###

    os.environ['PYTHONHASHSEED'] = str(33)
    np.random.seed(33)
    random.seed(33)

    left_input = []
    right_input = []
    targets = []
    
    for i in range(len(data)):
        if labels[i] == 1:
            pairs = 30
        else:
            pairs = 1
        for _ in range(pairs):
            compare_to = i
            while compare_to == i:
                compare_to = random.randint(0,len(data)-1)
            left_input.append(data[i])
            right_input.append(data[compare_to])
            # targets.append(euclidean(data[i], data[compare_to]))
            if labels[i] == 1 and labels[compare_to] == 1:
                targets.append(0.)
            elif labels[i] == 1 or labels[compare_to] == 1:
                targets.append(1.)
            else:
                targets.append(0.)

    left_input = np.asarray(left_input).reshape(-1, window_size, n_features)
    right_input = np.asarray(right_input).reshape(-1, window_size, n_features)
    targets = np.asarray(targets)
    return left_input, right_input, targets

def set_seed(seed):
    
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    random.seed(seed)
    

def SiamesNet():
    
    set_seed(33)
    
    left_input = Input((window_size, n_features))
    right_input = Input((window_size, n_features))

    lstmnet = Sequential([
        BatchNormalization(),
        LSTM(64, activation='relu', return_sequences=True, 
             input_shape=(window_size, n_features)),
        LSTM(16, activation='relu', return_sequences=True, 
             input_shape=(window_size, n_features)),
        LSTM(8, activation='relu')
    ])

    encoded_l = lstmnet(left_input)
    encoded_r = lstmnet(right_input)

    L1_layer = Lambda(lambda tensor: K.abs(tensor[0] - tensor[1]))

    L1_distance = L1_layer([encoded_l, encoded_r])
    drop = Dropout(0.2)(L1_distance)
    prediction = Dense(1,activation='sigmoid')(drop)
    model = Model(inputs=[left_input,right_input],outputs=prediction)
    
    return model

### Pre-processing

In [49]:
dfs = []
for i in range(1, 68):
    dfs.append(pd.read_csv('./data/A1Benchmark/real_{}.csv'.format(i), index_col='timestamp'))

In [69]:
window_size = 6
step = window_size
n_features = 1
windows = split_sequence(dfs[8].value, window_size, step)
labels = split_sequence(dfs[8].is_anomaly, window_size, step)
labels = np.array([1. if 1 in label_list else 0. for label_list in labels])

In [70]:
train_windows, test_windows, train_labels, test_labels = train_test_split(windows, labels, random_state=33)

In [71]:
train_scaler, test_scaler = MinMaxScaler(), MinMaxScaler()
train_windows = train_scaler.fit_transform(train_windows)
test_windows = test_scaler.fit_transform(test_windows)

In [72]:
train_left_input, train_right_input, train_targets = generate_pairs(train_windows, train_labels, window_size, n_features)
test_left_input, test_right_input, test_targets = generate_pairs(test_windows, test_labels, window_size, n_features)

### Modeling

In [73]:
siamese_net = SiamesNet()
siamese_net.compile(loss="binary_crossentropy", optimizer=Adam(learning_rate=1e-3), metrics=['accuracy'])

siamese_net.fit([train_left_input, train_right_input], train_targets, validation_split=0.15, epochs=50)

Epoch 1/50
8/8 [==============================] - 5s 91ms/step - loss: 0.6920 - accuracy: 0.6798 - val_loss: 0.6919 - val_accuracy: 0.5610
Epoch 2/50
8/8 [==============================] - 0s 10ms/step - loss: 0.6885 - accuracy: 0.7632 - val_loss: 0.6901 - val_accuracy: 0.5610
Epoch 3/50
8/8 [==============================] - 0s 11ms/step - loss: 0.6842 - accuracy: 0.7719 - val_loss: 0.6862 - val_accuracy: 0.9268
Epoch 4/50
8/8 [==============================] - 0s 9ms/step - loss: 0.6756 - accuracy: 0.8070 - val_loss: 0.6791 - val_accuracy: 1.0000
Epoch 5/50
8/8 [==============================] - 0s 10ms/step - loss: 0.6629 - accuracy: 0.7412 - val_loss: 0.6642 - val_accuracy: 1.0000
Epoch 6/50
8/8 [==============================] - 0s 10ms/step - loss: 0.6262 - accuracy: 0.7588 - val_loss: 0.6208 - val_accuracy: 0.9756
Epoch 7/50
8/8 [==============================] - 0s 9ms/step - loss: 0.5721 - accuracy: 0.7763 - val_loss: 0.5921 - val_accuracy: 1.0000
Epoch 8/50
8/8 [=============

### Predictions

In [74]:
y_pred = siamese_net.predict([test_left_input, test_right_input])

4/4 [==============================] - 1s 5ms/step


In [75]:
pred = ['fault' if i >= 0.99 else 'nofault' for i in y_pred]
real = ['fault' if i == 1 else 'nofault' for i in test_targets]

In [76]:
# os.environ['PYTHONHASHSEED'] = str(33)
# np.random.seed(33)
# random.seed(33)

# real = []
# pred = []
# rep = 4

# for j,i in enumerate(tqdm.tqdm(test_labels)):
    
#     test = test_windows[j]
    
#     nofault = np.random.choice(np.where(train_labels == 0)[0], rep)
#     nofault = train_windows[nofault]
#     nofault_sim = np.max([float(siamese_net.predict([test[np.newaxis,:],s[np.newaxis,:]], verbose=0)) for s in nofault])

#     fault = np.random.choice(np.where(train_labels == 1)[0], rep)
#     fault = train_windows[fault]
#     fault_sim = np.max([float(siamese_net.predict([test[np.newaxis,:],s[np.newaxis,:]], verbose=0)) for s in fault])
    
#     pred.append('nofault' if nofault_sim > fault_sim else 'fault')
#     real.append('nofault' if i == 0 else 'fault')
#     print(i)

In [77]:
print(classification_report(real, pred))

              precision    recall  f1-score   support

       fault       0.95      1.00      0.98        60
     nofault       1.00      0.95      0.97        59

    accuracy                           0.97       119
   macro avg       0.98      0.97      0.97       119
weighted avg       0.98      0.97      0.97       119

